# Notebook 3 : Analyse pour la Construction de la Couche Gold

## Objectif

Ce notebook vous guide dans l'**analyse des données de la couche silver** pour préparer la construction de la couche **gold** (analytics/BI/ML).

La couche gold contient des données transformées, nettoyées et optimisées pour l'analyse métier et la création de rapports/dashboards.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Exécuté le notebook `2_[LOAD]_load_to_bigquery.ipynb`** pour avoir toutes les tables dans BigQuery (dataset `silver`)
2. **Fichier `.env` configuré** avec les variables d'environnement nécessaires
3. **Service Account** avec les permissions BigQuery (`BigQuery Data Viewer`, `BigQuery Job User`)
4. **Packages Python installés** : `google-cloud-bigquery`, `pandas`, etc.

## Structure du Notebook

Ce notebook contient **4 tâches principales** à réaliser :

1. **Tâche 1 : Analyser la Granularité** - Comprendre le niveau de détail de chaque table
2. **Tâche 2 : Identifier les Transformations** - Déterminer les transformations nécessaires pour la couche gold
3. **Tâche 3 : Identifier les Clés de Jointure** - Mapper les relations entre les tables
4. **Tâche 4 : Analyse métier Identifier les **KPIs** métiers - 

## Résultats Attendus

À la fin de ce notebook, vous devriez avoir :
- Une compréhension claire de la structure et de la granularité de chaque table
- Une liste des transformations à appliquer pour créer la couche gold
- Un schéma de jointures documenté

## Configuration et Connexion à BigQuery

Cette section configure l'environnement et établit la connexion avec BigQuery pour explorer les données de la couche silver.


In [39]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

# Configuration
load_dotenv()

ROOT = Path.cwd().parent
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DATASET_ID = "silver"

# Authentification
creds = service_account.Credentials.from_service_account_file(SA_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print(f"[OK] - Connecté au projet: {PROJECT_ID}")
print(f"[OK] - Dataset: {DATASET_ID}")


[OK] - Connecté au projet: idfm-etl-reims-0224dy2025dy
[OK] - Dataset: silver


---

## Tâche 1 : Analyser la Granularité de Chaque Table

### Objectif

La **granularité** d'une table correspond au niveau de détail des données qu'elle contient. Comprendre la granularité est essentiel pour :
- Déterminer comment agréger les données
- Identifier les duplications potentielles
- Comprendre le niveau de détail nécessaire pour les analyses métier

### Instructions

Pour chaque table du dataset `silver`, vous devez :

1. **Lister les colonnes** et leurs types
2. **Identifier les clés primaires** ou les colonnes qui identifient de manière unique une ligne
3. **Déterminer la granularité** : à quel niveau de détail sont les données ?
   - Exemple : `fact_validations` pourrait être au niveau **jour × gare × type de titre**
4. **Compter les lignes** et estimer la taille des données
5. **Identifier les colonnes de dimension** (références vers d'autres tables)

### Exemple de Format de Réponse

```
Table: dim_gare
- Granularité: 1 ligne = 1 gare
- Clé primaire: id_gares
- Nombre de lignes: 1234
- Colonnes de dimension: aucune (table de dimension)
```


### À Compléter : Analyse de Granularité

**Tables de Dimension :**

1. `dim_gare`
2. `dim_ligne`
3. `dim_arret`
4. `dim_vacances_scolaires`
5. `dim_transporteur`

**Tables de Fait :**

6. `fact_validations_*` (toutes les tables de validation)

**Votre tâche :** Exécutez des requêtes SQL pour analyser chaque table et remplir le tableau ci-dessous.


In [40]:
from google.cloud import bigquery
import pandas as pd

def analyze_table(table_name, pk_cols=None):
    full_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
    table = bq_client.get_table(full_id)

    print(f"\n🧾 TABLE: {table_name}")
    print(f"📌 Nb lignes (metadata): {table.num_rows}")
    print(f"💾 Taille approx (bytes): {table.num_bytes}")

    print("\n=== Schéma ===")
    for field in table.schema:
        print(f"- {field.name}: {field.field_type} ({field.mode})")

    id_cols = [f.name for f in table.schema if f.name.startswith("id_")]
    print("\n=== Colonnes id_* ===")
    print(id_cols)

    if pk_cols:
        pk_concat = " || '|' || ".join([f"IFNULL(CAST({c} AS STRING), 'NULL')" for c in pk_cols])

        query = f"""
        SELECT
          COUNT(*) AS nb_lignes,
          COUNT(DISTINCT ({pk_concat})) AS nb_pk_uniques,
          {", ".join([f"SUM(CASE WHEN {c} IS NULL THEN 1 ELSE 0 END) AS nb_null_{c}" for c in pk_cols])}
        FROM `{full_id}`
        """
        df = bq_client.query(query).to_dataframe()
        print("\n=== Contrôle PK ===")
        display(df)

        query_dups = f"""
        SELECT {", ".join(pk_cols)}, COUNT(*) AS n
        FROM `{full_id}`
        GROUP BY {", ".join(pk_cols)}
        HAVING COUNT(*) > 1
        ORDER BY n DESC
        LIMIT 20
        """
        dups = bq_client.query(query_dups).to_dataframe()
        print("\n=== Doublons PK (top 20) ===")
        display(dups)


In [56]:
dataset_ref = bigquery.DatasetReference(PROJECT_ID, DATASET_ID)
tables = list(bq_client.list_tables(dataset_ref))

print("DATASET:", f"{PROJECT_ID}.{DATASET_ID}")
print("Nb tables:", len(tables))
for t in tables:
    print("-", t.table_id)


DATASET: idfm-etl-reims-0224dy2025dy.silver
Nb tables: 22
- dim_arret
- dim_ligne
- dim_transporteur
- dim_vacances_scolaires
- fact_validations_2015s1_nb_fer_csv
- fact_validations_2015s2_nb_fer_csv
- fact_validations_2016s1_nb_fer_txt
- fact_validations_2016s2_nb_fer_txt
- fact_validations_2017_s2_nb_fer_txt
- fact_validations_2017s1_nb_fer_txt
- fact_validations_2018_s1_nb_fer_txt
- fact_validations_2019_s1_nb_fer_txt
- fact_validations_2019_s2_nb_fer_txt
- fact_validations_2020_s1_nb_fer_txt
- fact_validations_2020_s2_nb_fer_txt
- fact_validations_2021_s1_nb_fer_txt
- fact_validations_2021_s2_nb_fer_txt
- fact_validations_2022_s1_nb_fer_txt
- fact_validations_2022_s2_nb_fer_txt
- fact_validations_2023_s2_nb_fer_txt
- fact_validations_2024_s1_nb_fer_txt
- gares


In [ ]:
#Analyser les tables “dimension

analyze_table("gares")



🧾 TABLE: gares
📌 Nb lignes (metadata): 1240
💾 Taille approx (bytes): 646069

=== Schéma ===
- geo_point_2d: GEOGRAPHY (NULLABLE)
- geo_shape: GEOGRAPHY (NULLABLE)
- id_gares: INTEGER (REQUIRED)
- nom_gares: STRING (NULLABLE)
- nom_so_gar: STRING (NULLABLE)
- nom_su_gar: STRING (NULLABLE)
- id_ref_zdc: INTEGER (NULLABLE)
- nom_zdc: STRING (NULLABLE)
- id_ref_zda: INTEGER (NULLABLE)
- nom_zda: STRING (NULLABLE)
- idrefliga: STRING (NULLABLE)
- idrefligc: STRING (NULLABLE)
- res_com: STRING (NULLABLE)
- indice_lig: STRING (NULLABLE)
- mode: STRING (NULLABLE)
- tertrain: STRING (NULLABLE)
- terrer: STRING (NULLABLE)
- termetro: STRING (NULLABLE)
- tertram: STRING (NULLABLE)
- terval: STRING (NULLABLE)
- exploitant: STRING (NULLABLE)
- idf: INTEGER (NULLABLE)
- principal: INTEGER (NULLABLE)
- x: FLOAT (NULLABLE)
- y: FLOAT (NULLABLE)
- picto: STRING (NULLABLE)
- nom_iv: STRING (NULLABLE)

=== Colonnes id_* ===
['id_gares', 'id_ref_zdc', 'id_ref_zda']

🧾 TABLE: dim_ligne
📌 Nb lignes (metada

In [ ]:
#Table : gares
#Granularité : 1 ligne = 1 gare
#Clé primaire : id_gares
#Nombre de lignes : 1240
#Colonnes de dimension : id_ref_zdc, id_ref_zda

In [41]:
analyze_table("dim_ligne")




🧾 TABLE: dim_ligne
📌 Nb lignes (metadata): 2120
💾 Taille approx (bytes): 592443

=== Schéma ===
- id_line: STRING (NULLABLE)
- name_line: STRING (NULLABLE)
- shortname_line: STRING (NULLABLE)
- transportmode: STRING (NULLABLE)
- transportsubmode: STRING (NULLABLE)
- type: STRING (NULLABLE)
- operatorref: STRING (NULLABLE)
- operatorname: STRING (NULLABLE)
- additionaloperators: STRING (NULLABLE)
- networkname: STRING (NULLABLE)
- colourweb_hexa: STRING (NULLABLE)
- textcolourweb_hexa: STRING (NULLABLE)
- colourprint_cmjn: STRING (NULLABLE)
- textcolourprint_hexa: STRING (NULLABLE)
- accessibility: STRING (NULLABLE)
- audiblesigns_available: STRING (NULLABLE)
- visualsigns_available: STRING (NULLABLE)
- id_groupoflines: STRING (NULLABLE)
- shortname_groupoflines: STRING (NULLABLE)
- notice_title: STRING (NULLABLE)
- notice_text: STRING (NULLABLE)
- picto: STRING (NULLABLE)
- valid_fromdate: DATE (NULLABLE)
- valid_todate: DATE (NULLABLE)
- status: STRING (NULLABLE)
- privatecode: STRIN

In [42]:
analyze_table("dim_arret")



🧾 TABLE: dim_arret
📌 Nb lignes (metadata): 38333
💾 Taille approx (bytes): 6596297

=== Schéma ===
- arrid: STRING (NULLABLE)
- arrversion: STRING (NULLABLE)
- arrcreated: TIMESTAMP (NULLABLE)
- arrchanged: TIMESTAMP (NULLABLE)
- arrname: STRING (NULLABLE)
- arrtype: STRING (NULLABLE)
- arrxepsg2154: INTEGER (NULLABLE)
- arryepsg2154: INTEGER (NULLABLE)
- arrtown: STRING (NULLABLE)
- arrpostalregion: STRING (NULLABLE)
- arraccessibility: STRING (NULLABLE)
- arraudiblesignals: STRING (NULLABLE)
- arrvisualsigns: STRING (NULLABLE)
- arrfarezone: STRING (NULLABLE)
- zdaid: STRING (NULLABLE)
- arrgeopoint: GEOGRAPHY (NULLABLE)

=== Colonnes id_* ===
[]


In [43]:
analyze_table("dim_vacances_scolaires")



🧾 TABLE: dim_vacances_scolaires
📌 Nb lignes (metadata): 2306
💾 Taille approx (bytes): 170029

=== Schéma ===
- description: STRING (NULLABLE)
- population: STRING (NULLABLE)
- start_date: TIMESTAMP (NULLABLE)
- end_date: TIMESTAMP (NULLABLE)
- location: STRING (NULLABLE)
- zones: STRING (NULLABLE)
- annee_scolaire: STRING (NULLABLE)

=== Colonnes id_* ===
[]


In [44]:
analyze_table("dim_transporteur")


🧾 TABLE: dim_transporteur
📌 Nb lignes (metadata): 53
💾 Taille approx (bytes): 14726

=== Schéma ===
- operatorname: STRING (NULLABLE)
- operatorref: STRING (NULLABLE)
- housenumber: STRING (NULLABLE)
- street: STRING (NULLABLE)
- addressline1: STRING (NULLABLE)
- town: STRING (NULLABLE)
- postcode: STRING (NULLABLE)
- postcodeextension: STRING (NULLABLE)
- phone: STRING (NULLABLE)
- url: STRING (NULLABLE)
- furtherdetails: STRING (NULLABLE)
- contactperson: STRING (NULLABLE)
- logo: STRING (NULLABLE)
- email: STRING (NULLABLE)

=== Colonnes id_* ===
[]


In [45]:
analyze_table("dim_arret", pk_cols=["arrid"])


🧾 TABLE: dim_arret
📌 Nb lignes (metadata): 38333
💾 Taille approx (bytes): 6596297

=== Schéma ===
- arrid: STRING (NULLABLE)
- arrversion: STRING (NULLABLE)
- arrcreated: TIMESTAMP (NULLABLE)
- arrchanged: TIMESTAMP (NULLABLE)
- arrname: STRING (NULLABLE)
- arrtype: STRING (NULLABLE)
- arrxepsg2154: INTEGER (NULLABLE)
- arryepsg2154: INTEGER (NULLABLE)
- arrtown: STRING (NULLABLE)
- arrpostalregion: STRING (NULLABLE)
- arraccessibility: STRING (NULLABLE)
- arraudiblesignals: STRING (NULLABLE)
- arrvisualsigns: STRING (NULLABLE)
- arrfarezone: STRING (NULLABLE)
- zdaid: STRING (NULLABLE)
- arrgeopoint: GEOGRAPHY (NULLABLE)

=== Colonnes id_* ===
[]


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_arrid
0,38333,38333,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,arrid,n


In [46]:
#Granularité : 1 ligne = 1 arrêt

#PK : arrid (clé primaire simple)

#Doublons : aucun

In [21]:
analyze_table("gares", pk_cols=["id_gares"])



🧾 TABLE: gares
📌 Nb lignes (metadata): 1240
💾 Taille approx (bytes): 646069

=== Schéma ===
- geo_point_2d: GEOGRAPHY (NULLABLE)
- geo_shape: GEOGRAPHY (NULLABLE)
- id_gares: INTEGER (REQUIRED)
- nom_gares: STRING (NULLABLE)
- nom_so_gar: STRING (NULLABLE)
- nom_su_gar: STRING (NULLABLE)
- id_ref_zdc: INTEGER (NULLABLE)
- nom_zdc: STRING (NULLABLE)
- id_ref_zda: INTEGER (NULLABLE)
- nom_zda: STRING (NULLABLE)
- idrefliga: STRING (NULLABLE)
- idrefligc: STRING (NULLABLE)
- res_com: STRING (NULLABLE)
- indice_lig: STRING (NULLABLE)
- mode: STRING (NULLABLE)
- tertrain: STRING (NULLABLE)
- terrer: STRING (NULLABLE)
- termetro: STRING (NULLABLE)
- tertram: STRING (NULLABLE)
- terval: STRING (NULLABLE)
- exploitant: STRING (NULLABLE)
- idf: INTEGER (NULLABLE)
- principal: INTEGER (NULLABLE)
- x: FLOAT (NULLABLE)
- y: FLOAT (NULLABLE)
- picto: STRING (NULLABLE)
- nom_iv: STRING (NULLABLE)

=== Colonnes id_* ===
['id_gares', 'id_ref_zdc', 'id_ref_zda']

=== Contrôle PK ===


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes,nb_pk_uniques,nb_null_id_gares
0,1240,1237,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,id_gares,n
0,267,2
1,1252,2
2,1855,2


In [ ]:

#Donc La table gares présente des doublons sur id_gares car une même gare peut 
# exister pour plusieurs exploitants / modes. La clé n’est donc pas strictement 
# unique à ce stade.

In [47]:
analyze_table("dim_ligne", pk_cols=["id_line"])



🧾 TABLE: dim_ligne
📌 Nb lignes (metadata): 2120
💾 Taille approx (bytes): 592443

=== Schéma ===
- id_line: STRING (NULLABLE)
- name_line: STRING (NULLABLE)
- shortname_line: STRING (NULLABLE)
- transportmode: STRING (NULLABLE)
- transportsubmode: STRING (NULLABLE)
- type: STRING (NULLABLE)
- operatorref: STRING (NULLABLE)
- operatorname: STRING (NULLABLE)
- additionaloperators: STRING (NULLABLE)
- networkname: STRING (NULLABLE)
- colourweb_hexa: STRING (NULLABLE)
- textcolourweb_hexa: STRING (NULLABLE)
- colourprint_cmjn: STRING (NULLABLE)
- textcolourprint_hexa: STRING (NULLABLE)
- accessibility: STRING (NULLABLE)
- audiblesigns_available: STRING (NULLABLE)
- visualsigns_available: STRING (NULLABLE)
- id_groupoflines: STRING (NULLABLE)
- shortname_groupoflines: STRING (NULLABLE)
- notice_title: STRING (NULLABLE)
- notice_text: STRING (NULLABLE)
- picto: STRING (NULLABLE)
- valid_fromdate: DATE (NULLABLE)
- valid_todate: DATE (NULLABLE)
- status: STRING (NULLABLE)
- privatecode: STRIN

c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_id_line
0,2120,2120,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,id_line,n


In [48]:
analyze_table("dim_transporteur", pk_cols=["operatorref"])



🧾 TABLE: dim_transporteur
📌 Nb lignes (metadata): 53
💾 Taille approx (bytes): 14726

=== Schéma ===
- operatorname: STRING (NULLABLE)
- operatorref: STRING (NULLABLE)
- housenumber: STRING (NULLABLE)
- street: STRING (NULLABLE)
- addressline1: STRING (NULLABLE)
- town: STRING (NULLABLE)
- postcode: STRING (NULLABLE)
- postcodeextension: STRING (NULLABLE)
- phone: STRING (NULLABLE)
- url: STRING (NULLABLE)
- furtherdetails: STRING (NULLABLE)
- contactperson: STRING (NULLABLE)
- logo: STRING (NULLABLE)
- email: STRING (NULLABLE)

=== Colonnes id_* ===
[]


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_operatorref
0,53,53,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,operatorref,n


In [49]:
analyze_table("dim_vacances_scolaires", pk_cols=["start_date", "end_date", "location", "population"])



🧾 TABLE: dim_vacances_scolaires
📌 Nb lignes (metadata): 2306
💾 Taille approx (bytes): 170029

=== Schéma ===
- description: STRING (NULLABLE)
- population: STRING (NULLABLE)
- start_date: TIMESTAMP (NULLABLE)
- end_date: TIMESTAMP (NULLABLE)
- location: STRING (NULLABLE)
- zones: STRING (NULLABLE)
- annee_scolaire: STRING (NULLABLE)

=== Colonnes id_* ===
[]


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_start_date,nb_null_end_date,nb_null_location,nb_null_population
0,2306,2306,0,0,0,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,start_date,end_date,location,population,n


In [51]:
#Trouver la bonne PK pour gares

query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.gares`
WHERE id_gares IN (
  SELECT id_gares
  FROM `{PROJECT_ID}.{DATASET_ID}.gares`
  GROUP BY id_gares
  HAVING COUNT(*) > 1
)
ORDER BY id_gares
"""
df_dup = bq_client.query(query).to_dataframe()
display(df_dup)


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,geo_point_2d,geo_shape,id_gares,nom_gares,nom_so_gar,nom_su_gar,id_ref_zdc,nom_zdc,id_ref_zda,nom_zda,...,termetro,tertram,terval,exploitant,idf,principal,x,y,picto,nom_iv
0,POINT(2.81024822073006 48.9030729151079),POINT(2.81024822073006 48.9030729151079),267,Esbly,None,None,68653,Esbly,47322,Esbly,...,0,0,0,SNCF,1,1,686089.6334,6.867025e+06,"{""thumbnail"": true, ""filename"": ""train_P.svg"",...",Esbly
1,POINT(2.81024822073006 48.9030729151079),POINT(2.81024822073006 48.9030729151079),267,Esbly,None,None,68653,Esbly,492417,Esbly,...,0,TRAM 14,0,STRETTO,1,1,686089.6334,6.867025e+06,None,Esbly
2,POINT(2.38073560639509 48.9137105455496),POINT(2.3807356063951 48.9137105455496),1252,Mairie d'Aubervilliers,None,None,72524,Mairie d'Aubervilliers,483899,Mairie d'Aubervilliers,...,METRO 12,0,0,RATP,1,1,654612.5235,6.868369e+06,"{""thumbnail"": true, ""filename"": ""metro_12.svg""...",Mairie d'Aubervilliers
3,POINT(2.07474657726422 48.7990098108357),POINT(2.07474657726422 48.7990098108357),1252,Saint-Cyr,None,None,73731,Saint-Cyr,480909,Saint-Cyr,...,0,TRAM 13,0,TRANSKEO,1,0,631997.9595,6.855837e+06,None,Saint-Cyr
4,POINT(2.34936523945399 48.7930549126736),POINT(2.34936523945399 48.7930549126736),1855,Villejuif - Gustave Roussy,None,None,478860,Villejuif - Gustave Roussy,490907,Villejuif - Gustave Roussy,...,0,0,0,RATP,1,0,652202.8135,6.854972e+06,"{""thumbnail"": true, ""filename"": ""metro_14.svg""...",Villejuif - Gustave Roussy
5,POINT(2.45471421106498 48.8783367019385),POINT(2.45471421106498 48.8783367019385),1855,Montreuil - Hôpital,None,None,71897,Montreuil - Hôpital,490767,Montreuil - Hôpital,...,0,0,0,RATP,1,0,660007.3499,6.864395e+06,"{""thumbnail"": true, ""filename"": ""metro_11.svg""...",Montreuil - Hôpital


In [52]:
analyze_table("gares", pk_cols=["id_gares", "exploitant", "termetro", "tertram"])



🧾 TABLE: gares
📌 Nb lignes (metadata): 1240
💾 Taille approx (bytes): 646069

=== Schéma ===
- geo_point_2d: GEOGRAPHY (NULLABLE)
- geo_shape: GEOGRAPHY (NULLABLE)
- id_gares: INTEGER (REQUIRED)
- nom_gares: STRING (NULLABLE)
- nom_so_gar: STRING (NULLABLE)
- nom_su_gar: STRING (NULLABLE)
- id_ref_zdc: INTEGER (NULLABLE)
- nom_zdc: STRING (NULLABLE)
- id_ref_zda: INTEGER (NULLABLE)
- nom_zda: STRING (NULLABLE)
- idrefliga: STRING (NULLABLE)
- idrefligc: STRING (NULLABLE)
- res_com: STRING (NULLABLE)
- indice_lig: STRING (NULLABLE)
- mode: STRING (NULLABLE)
- tertrain: STRING (NULLABLE)
- terrer: STRING (NULLABLE)
- termetro: STRING (NULLABLE)
- tertram: STRING (NULLABLE)
- terval: STRING (NULLABLE)
- exploitant: STRING (NULLABLE)
- idf: INTEGER (NULLABLE)
- principal: INTEGER (NULLABLE)
- x: FLOAT (NULLABLE)
- y: FLOAT (NULLABLE)
- picto: STRING (NULLABLE)
- nom_iv: STRING (NULLABLE)

=== Colonnes id_* ===
['id_gares', 'id_ref_zdc', 'id_ref_zda']


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_id_gares,nb_null_exploitant,nb_null_termetro,nb_null_tertram
0,1240,1239,0,0,0,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,id_gares,exploitant,termetro,tertram,n
0,1855,RATP,0,0,2


In [53]:
query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.gares`
WHERE STRUCT(id_gares, exploitant, termetro, tertram) IN (
  SELECT AS STRUCT id_gares, exploitant, termetro, tertram
  FROM `{PROJECT_ID}.{DATASET_ID}.gares`
  GROUP BY id_gares, exploitant, termetro, tertram
  HAVING COUNT(*) > 1
)
ORDER BY id_gares, exploitant, termetro, tertram
"""
df = bq_client.query(query).to_dataframe()
display(df)


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,geo_point_2d,geo_shape,id_gares,nom_gares,nom_so_gar,nom_su_gar,id_ref_zdc,nom_zdc,id_ref_zda,nom_zda,...,termetro,tertram,terval,exploitant,idf,principal,x,y,picto,nom_iv
0,POINT(2.34936523945399 48.7930549126736),POINT(2.34936523945399 48.7930549126736),1855,Villejuif - Gustave Roussy,None,None,478860,Villejuif - Gustave Roussy,490907,Villejuif - Gustave Roussy,...,0,0,0,RATP,1,0,652202.8135,6.854972e+06,"{""thumbnail"": true, ""filename"": ""metro_14.svg""...",Villejuif - Gustave Roussy
1,POINT(2.45471421106498 48.8783367019385),POINT(2.45471421106498 48.8783367019385),1855,Montreuil - Hôpital,None,None,71897,Montreuil - Hôpital,490767,Montreuil - Hôpital,...,0,0,0,RATP,1,0,660007.3499,6.864395e+06,"{""thumbnail"": true, ""filename"": ""metro_11.svg""...",Montreuil - Hôpital


In [54]:
analyze_table("gares", pk_cols=["id_gares", "nom_gares"])



🧾 TABLE: gares
📌 Nb lignes (metadata): 1240
💾 Taille approx (bytes): 646069

=== Schéma ===
- geo_point_2d: GEOGRAPHY (NULLABLE)
- geo_shape: GEOGRAPHY (NULLABLE)
- id_gares: INTEGER (REQUIRED)
- nom_gares: STRING (NULLABLE)
- nom_so_gar: STRING (NULLABLE)
- nom_su_gar: STRING (NULLABLE)
- id_ref_zdc: INTEGER (NULLABLE)
- nom_zdc: STRING (NULLABLE)
- id_ref_zda: INTEGER (NULLABLE)
- nom_zda: STRING (NULLABLE)
- idrefliga: STRING (NULLABLE)
- idrefligc: STRING (NULLABLE)
- res_com: STRING (NULLABLE)
- indice_lig: STRING (NULLABLE)
- mode: STRING (NULLABLE)
- tertrain: STRING (NULLABLE)
- terrer: STRING (NULLABLE)
- termetro: STRING (NULLABLE)
- tertram: STRING (NULLABLE)
- terval: STRING (NULLABLE)
- exploitant: STRING (NULLABLE)
- idf: INTEGER (NULLABLE)
- principal: INTEGER (NULLABLE)
- x: FLOAT (NULLABLE)
- y: FLOAT (NULLABLE)
- picto: STRING (NULLABLE)
- nom_iv: STRING (NULLABLE)

=== Colonnes id_* ===
['id_gares', 'id_ref_zdc', 'id_ref_zda']


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_id_gares,nb_null_nom_gares
0,1240,1239,0,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,id_gares,nom_gares,n
0,267,Esbly,2


In [55]:
analyze_table("gares", pk_cols=["id_gares", "nom_gares", "exploitant", "termetro", "tertram"])



🧾 TABLE: gares
📌 Nb lignes (metadata): 1240
💾 Taille approx (bytes): 646069

=== Schéma ===
- geo_point_2d: GEOGRAPHY (NULLABLE)
- geo_shape: GEOGRAPHY (NULLABLE)
- id_gares: INTEGER (REQUIRED)
- nom_gares: STRING (NULLABLE)
- nom_so_gar: STRING (NULLABLE)
- nom_su_gar: STRING (NULLABLE)
- id_ref_zdc: INTEGER (NULLABLE)
- nom_zdc: STRING (NULLABLE)
- id_ref_zda: INTEGER (NULLABLE)
- nom_zda: STRING (NULLABLE)
- idrefliga: STRING (NULLABLE)
- idrefligc: STRING (NULLABLE)
- res_com: STRING (NULLABLE)
- indice_lig: STRING (NULLABLE)
- mode: STRING (NULLABLE)
- tertrain: STRING (NULLABLE)
- terrer: STRING (NULLABLE)
- termetro: STRING (NULLABLE)
- tertram: STRING (NULLABLE)
- terval: STRING (NULLABLE)
- exploitant: STRING (NULLABLE)
- idf: INTEGER (NULLABLE)
- principal: INTEGER (NULLABLE)
- x: FLOAT (NULLABLE)
- y: FLOAT (NULLABLE)
- picto: STRING (NULLABLE)
- nom_iv: STRING (NULLABLE)

=== Colonnes id_* ===
['id_gares', 'id_ref_zdc', 'id_ref_zda']


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_id_gares,nb_null_nom_gares,nb_null_exploitant,nb_null_termetro,nb_null_tertram
0,1240,1240,0,0,0,0,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,id_gares,nom_gares,exploitant,termetro,tertram,n


---

## Tâche 2 : Identifier les Transformations Nécessaires

### Objectif

Identifier les **transformations** à appliquer aux données de la couche silver pour créer la couche gold optimisée pour l'analyse.

### Types de Transformations Possibles

1. **Nettoyage des données**
   - Suppression des doublons
   - Gestion des valeurs NULL
   - Normalisation des formats (dates, textes)

2. **Enrichissement**
   - Ajout de colonnes calculées
   - Jointures avec les tables de dimension
   - Ajout de catégories/segments

3. **Agrégation**
   - Regroupement par dimensions (jour, gare, ligne, etc.)
   - Calcul de métriques (somme, moyenne, comptage)
   - Création de tables pré-agrégées

### Instructions

Pour chaque table, identifiez :
1. **Les problèmes de qualité** à corriger
2. **Les transformations nécessaires** avec des exemples concrets
3. **Les colonnes à ajouter** (calculées ou issues de jointures)
4. **Les agrégations possibles** pour optimiser les requêtes


In [63]:
# Exemple 1 : Vérifier les doublons dans fact_validations
analyze_table("fact_validations_2024_s1_nb_fer_txt")



🧾 TABLE: fact_validations_2024_s1_nb_fer_txt
📌 Nb lignes (metadata): 859043
💾 Taille approx (bytes): 66692434

=== Schéma ===
- JOUR: DATE (NULLABLE)
- CODE_STIF_TRNS: INTEGER (NULLABLE)
- CODE_STIF_RES: INTEGER (NULLABLE)
- CODE_STIF_ARRET: INTEGER (NULLABLE)
- LIBELLE_ARRET: STRING (NULLABLE)
- ID_ZDC: INTEGER (NULLABLE)
- CATEGORIE_TITRE: STRING (NULLABLE)
- NB_VALD: INTEGER (NULLABLE)

=== Colonnes id_* ===
[]


In [64]:
table_name = "fact_validations_2024_s1_nb_fer_txt"
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.{table_name}")
cols = [f.name for f in table.schema]
cols


['JOUR',
 'CODE_STIF_TRNS',
 'CODE_STIF_RES',
 'CODE_STIF_ARRET',
 'LIBELLE_ARRET',
 'ID_ZDC',
 'CATEGORIE_TITRE',
 'NB_VALD']

In [65]:
table_name = "fact_validations_2024_s1_nb_fer_txt"

query = f"""
SELECT
  JOUR,
  CODE_STIF_ARRET,
  CATEGORIE_TITRE,
  COUNT(*) AS n
FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}`
GROUP BY JOUR, CODE_STIF_ARRET, CATEGORIE_TITRE
HAVING COUNT(*) > 1
ORDER BY n DESC
LIMIT 20
"""
df_duplicates = bq_client.query(query).to_dataframe()
display(df_duplicates)


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,JOUR,CODE_STIF_ARRET,CATEGORIE_TITRE,n
0,2024-02-21,48093,Amethyste,5
1,2024-01-29,48093,Amethyste,5
2,2024-02-13,48093,Amethyste,5
3,2024-01-10,48093,Amethyste,5
4,2024-01-23,48093,Amethyste,5
5,2024-02-26,48093,Amethyste,5
6,2024-01-22,48093,Amethyste,5
7,2024-01-24,48093,Amethyste,5
8,2024-02-06,48093,Amethyste,5
9,2024-02-05,48093,Amethyste,5


In [66]:
analyze_table(
    "fact_validations_2024_s1_nb_fer_txt",
    pk_cols=["JOUR", "CODE_STIF_ARRET", "CATEGORIE_TITRE"]
)



🧾 TABLE: fact_validations_2024_s1_nb_fer_txt
📌 Nb lignes (metadata): 859043
💾 Taille approx (bytes): 66692434

=== Schéma ===
- JOUR: DATE (NULLABLE)
- CODE_STIF_TRNS: INTEGER (NULLABLE)
- CODE_STIF_RES: INTEGER (NULLABLE)
- CODE_STIF_ARRET: INTEGER (NULLABLE)
- LIBELLE_ARRET: STRING (NULLABLE)
- ID_ZDC: INTEGER (NULLABLE)
- CATEGORIE_TITRE: STRING (NULLABLE)
- NB_VALD: INTEGER (NULLABLE)

=== Colonnes id_* ===
[]


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Contrôle PK ===


,nb_lignes,nb_pk_uniques,nb_null_JOUR,nb_null_CODE_STIF_ARRET,nb_null_CATEGORIE_TITRE
0,859043,853709,0,905,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,JOUR,CODE_STIF_ARRET,CATEGORIE_TITRE,n
0,2024-02-21,48093,Amethyste,5
1,2024-01-29,48093,Amethyste,5
2,2024-02-13,48093,Amethyste,5
3,2024-01-10,48093,Amethyste,5
4,2024-01-23,48093,Amethyste,5
5,2024-02-26,48093,Amethyste,5
6,2024-01-22,48093,Amethyste,5
7,2024-01-24,48093,Amethyste,5
8,2024-02-06,48093,Amethyste,5
9,2024-02-05,48093,Amethyste,5


In [67]:
query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2024_s1_nb_fer_txt`
WHERE JOUR = DATE '2024-02-21'
  AND CODE_STIF_ARRET = 48093
  AND CATEGORIE_TITRE = 'Amethyste'
"""
df = bq_client.query(query).to_dataframe()
display(df)


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,ID_ZDC,CATEGORIE_TITRE,NB_VALD
0,2024-02-21,800,800,48093,Allée Royale T13,64049,Amethyste,2
1,2024-02-21,800,800,48093,Bailly T13,480951,Amethyste,7
2,2024-02-21,800,800,48093,L'Etang - Les Sablons T13,480950,Amethyste,1
3,2024-02-21,800,800,48093,Les Portes de Saint-Cyr T13,480952,Amethyste,31
4,2024-02-21,800,800,48093,Noisy-le-Roi T13,64246,Amethyste,10


In [68]:
#cles logiaue correcte
analyze_table(
    "fact_validations_2024_s1_nb_fer_txt",
    pk_cols=["JOUR", "CODE_STIF_ARRET", "ID_ZDC", "CATEGORIE_TITRE"]
)



🧾 TABLE: fact_validations_2024_s1_nb_fer_txt
📌 Nb lignes (metadata): 859043
💾 Taille approx (bytes): 66692434

=== Schéma ===
- JOUR: DATE (NULLABLE)
- CODE_STIF_TRNS: INTEGER (NULLABLE)
- CODE_STIF_RES: INTEGER (NULLABLE)
- CODE_STIF_ARRET: INTEGER (NULLABLE)
- LIBELLE_ARRET: STRING (NULLABLE)
- ID_ZDC: INTEGER (NULLABLE)
- CATEGORIE_TITRE: STRING (NULLABLE)
- NB_VALD: INTEGER (NULLABLE)

=== Colonnes id_* ===
[]

=== Contrôle PK ===


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes,nb_pk_uniques,nb_null_JOUR,nb_null_CODE_STIF_ARRET,nb_null_ID_ZDC,nb_null_CATEGORIE_TITRE
0,859043,859035,0,905,0,0


c:\Users\Claudy LINCY\Desktop\M2_SEP\Outils Big data\m2-univ-reims-sep-cs-etl-sncf-gcp\venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



=== Doublons PK (top 20) ===


,JOUR,CODE_STIF_ARRET,ID_ZDC,CATEGORIE_TITRE,n
0,2024-03-21,<NA>,999999,Forfait Navigo,2
1,2024-03-12,<NA>,999999,NON DEFINI,2
2,2024-03-12,<NA>,999999,Forfait Navigo,2
3,2024-03-20,<NA>,999999,Contrat Solidarité Transport,2
4,2024-03-12,<NA>,999999,Contrat Solidarité Transport,2
5,2024-03-11,<NA>,999999,NON DEFINI,2
6,2024-03-20,<NA>,999999,Forfait Navigo,2
7,2024-03-11,<NA>,999999,Forfait Navigo,2


In [ ]:
#La quasi-totalité des lignes est unique selon la clé
#(JOUR, CODE_STIF_ARRET, ID_ZDC, CATEGORIE_TITRE).
#Les rares doublons restants correspondent à des lignes agrégées
#sans arrêt précis (CODE_STIF_ARRET nul, ID_ZDC = 999999),
#utilisées pour des catégories globales de titres.
#Ces cas sont identifiés comme des exceptions métiers et ne remettent
#pas en cause la granularité principale de la table.

---

## Tâche 3 : Identifier les Clés de Jointure Possibles

### Objectif

Identifier toutes les **relations possibles** entre les tables pour pouvoir créer des jointures dans la couche gold.

### Instructions

Pour chaque paire de tables, identifiez :

1. **Les colonnes de jointure** (clés étrangères)
2. **Le type de relation** (1-1, 1-N, N-N)
3. **La cardinalité** (combien de lignes de la table A correspondent à combien de lignes de la table B)
4. **Vérifier l'intégrité référentielle** (toutes les clés étrangères existent-elles dans la table de dimension ?)


---

## Tâche 4 : Analyse métier

Identifier les **KPIs** (Key Performance Indicators) à calculer :
- Nombre total de validations par période
- Répartition par type de titre
- Top 10 des gares les plus fréquentées
- Comparaison jour ouvrable vs weekend

